# Sensor placement

[Simon Dobson](mailto:simon.dobson@st-andrews.ac.uk) <br>
School of Computer Science, University of St Andrews, Scotland UK

In [1]:
import json
from datetime import datetime, timedelta
from itertools import combinations
import numpy
import netCDF4 as nc
from pandas import DataFrame
from geopandas import GeoDataFrame, GeoSeries
from networkx import Graph

from pyproj import CRS, Transformer, Geod
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import folium
import folium.plugins
import shapely
import shapely.geometry
from shapely.ops import cascaded_union

### Import the CEH-GEAR dataset

The CEH_GEAR dataset is the "reference" datase for rainfall interpolated at 1km squares across Great Britain. 

In [2]:
days_base = datetime(year=1800, month=1, day=1)

In [3]:
def daysToDate(days):
    '''Return the date of the given day offset'''
    return days_base + timedelta(days=days) 

def dateToDays(date):
    '''Return the days offset to the given date.'''
    return date - days_base

def dateToIndex(ds, date):
    '''Return the index into the given dataset of the given date.'''
    return dateToDays(date).days - ds['time'][0]

In [4]:
ceh_filename='datasets/CEH_GEAR_monthly_GB_2017.nc'
ceh = nc.Dataset(ceh_filename)

In [262]:
xs, ys, ts, lats, lons = ceh['x'][:], ceh['y'][:], ceh['time'][:], ceh['lat'][:], ceh['lon'][:]
rainfall = ceh['rainfall_amount'][1, :, :]
distance = ceh['min_dist'][1, :, :]

The data is referenced by co-ordinates on the UK National Grid. Since we need to be able to also work in standard latitude-longitudfe co-ordinates, we construct a transformer between the two co-ordinate reference systems.  

In [21]:
uk_grid_crs = CRS.from_string('EPSG:27700')   # UK national grid
latlon_crs = CRS.from_string('EPSG:4326')     # global Mercator (WGS 84)

proj = Transformer.from_crs(uk_grid_crs, latlon_crs)

### Import the boundaries of counties

We construct a dataframe with the names and b oundary shapoes of counties (and other administrative areas of the UK) by reading the master shape file.

In [7]:
boundaries_filename = 'datasets/UK_BUC.geojson'
with open(boundaries_filename, 'r') as fh:
    counties_json = json.load(fh)

In [187]:
counties = GeoDataFrame(columns=['county', 'geometry'])
for c in counties_json['features']:
    counties.loc[len(counties.index)] = {'county': c['properties']['ctyua18nm'],
                                         'geometry': shapely.geometry.shape(c['geometry'])}

We'll focus on Fife for the rest of this notebook.

In [188]:
fife = counties[counties['county'] == 'Fife'].iloc[0]

We extract the boundary polygon for Fife.

In [189]:
fife_boundary = fife['geometry']

### Load the SEPA data

In [191]:
sepastations_filename = 'datasets/sepa-stations.json'
with open(sepastations_filename, 'r') as fh:
     sepastations_json = json.load(fh)

In [207]:
sepastations = GeoDataFrame(columns=['name', 'id', 'longitude', 'latitude', 'geometry'])
for i in range(len(sepastations_json)):
    s = sepastations_json[i]
    sepastations.loc[i] = {'id': int(s['station_no']),
                           'name': s['station_name'],
                           'longitude': float(s['station_longitude']),
                           'latitude': float(s['station_latitude']),
                           'geometry': shapely.geometry.Point(float(s['station_longitude']), float(s['station_latitude']))}

In [208]:
fife_stations = sepastations[sepastations['geometry'].within(fife_boundary)]

In [209]:
fife_stations

,name,id,longitude,latitude,geometry
9,Annfield,15198,-3.371633,56.066401,POINT (-3.37163 56.06640)
20,Baintown Rain-gauge,501947,-3.042679,56.221016,POINT (-3.04268 56.22102)
45,Cambo Sands,338380,-2.650067,56.300880,POINT (-2.65007 56.30088)
102,Fife Airport,15151,-3.222449,56.181650,POINT (-3.22245 56.18165)
157,Kinghorn Ecology Centre Rain Gauge,473550,-3.193433,56.074623,POINT (-3.19343 56.07462)
211,Newton of Falkland,15083,-3.186269,56.254315,POINT (-3.18627 56.25431)
235,Rossie Farm,15070,-3.202718,56.299126,POINT (-3.20272 56.29913)
237,Saline,15155,-3.585959,56.122240,POINT (-3.58596 56.12224)
249,St Monance,335620,-2.782714,56.204785,POINT (-2.78271 56.20479)
252,Strathkinness,11368,-2.863919,56.331369,POINT (-2.86392 56.33137)


### Displaying the reference data 

To display the rainfall data we extract from the CEH-GEAR dataset all the points that lie within the boundary of interest. We then normalise the rainfall into the range [0.0, 1.0] for simplicity.

In [264]:
fife_points = GeoDataFrame(columns=['i_east', 'i_north', 'longitude', 'latitude', 'geometry'])
for i in range(len(xs)):
    for j in range(len(ys)):
        if not rainfall.mask[j, i]:
            lat, lon = proj.transform(xs[i], ys[j])
            p = shapely.geometry.Point(lon, lat)
            if fife_boundary.contains(p):
                fife_points.loc[len(fife_points.index)] = {'i_east': i,
                                                           'i_north': j,
                                                           'longitude': lon,
                                                           'latitude': lat,
                                                           'geometry': p}

In [265]:
fife_points_rainfall = fife_points.copy()
fife_points_rainfall['rainfall'] = fife_points_rainfall.apply(lambda r: rainfall[r['i_north'], r['i_east']], axis=1)

# normalise
r_max = max(fife_points_rainfall['rainfall'])
r_min = min(fife_points_rainfall['rainfall'])
r_step = 1 / (r_max - r_min)
fife_points_rainfall['normalised_rainfall'] = (fife_points_rainfall['rainfall'] - r_min) * r_step

We can then construct a map with overlays for the region of interest and the interpolated rainfall.

In [266]:
uk_ceh = folium.Map(location=proj.transform(350608, 716790))

# add the boundary of Fife
folium.GeoJson(fife['geometry']).add_to(uk_ceh)

# add the stations
for i in range(len(fife_stations)):
    s = fife_stations.iloc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W)').add_to(uk_ceh)

# add the heat map
_ = folium.plugins.HeatMap(data=fife_points_rainfall[['latitude', 'longitude', 'normalised_rainfall']], min_opacity=0.01, radius=20, blur=20, max_zoom=5).add_to(uk_ceh)

In [267]:
uk_ceh

Alternatively, if the actual observations at the stations themselves are of interst, we can plot them too. We first load the raw data.

In [183]:
sepa_filename='datasets/sepa_monthly_2017.nc'
sepa = nc.Dataset(sepa_filename)

In [210]:
uk_stations = folium.Map(location=proj.transform(350608, 716790))
folium.GeoJson(fife['geometry']).add_to(uk_stations)

# add the station obsertaions
for i in range(len(fife_stations)):
    s = fife_stations.iloc[i]
    name, id, lon, lat = s['name'], s['id'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(id)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} {rain:.2f}mm').add_to(uk_stations)

In [211]:
uk_stations

### Building a network of stations

We might also want to construct a neighbourhood graph from the stations. For this we assume that stations are the nodes of a network, with edges between nodes that are within a certain distance of each other.

In [212]:
g_fife = Graph()

# construct nodes for each station
for i in range(len(fife_stations)):
    s = fife_stations.iloc[i]
    g_fife.add_node(i)
    g_fife.nodes[i]['name'] = s['name']
    g_fife.nodes[i]['id'] = s['id']
    g_fife.nodes[i]['longitude'] = s['longitude']
    g_fife.nodes[i]['latitude'] = s['latitude']

# form edges between nodes within 25km of each other
geodesic = Geod(ellps='clrk66')
for n, m in combinations(list(g_fife.nodes()), 2):
    lon1, lat1 = g_fife.nodes[n]['longitude'], g_fife.nodes[n]['latitude']
    lon2, lat2 = g_fife.nodes[m]['longitude'], g_fife.nodes[m]['latitude']
    _, _, d = geodesic.inv(lon1, lat1, lon2, lat2)
    if d <= 25000:
        g_fife.add_edge(n, m)
        g_fife.edges[n, m]['distance'] = d

In [213]:
fife_stations_network = folium.Map(location=proj.transform(350608, 716790))
folium.GeoJson(fife['geometry']).add_to(fife_stations_network)

# add the stations
for n in g_fife.nodes():
    name, id, lat, lon = g_fife.nodes[n]['name'], g_fife.nodes[n]['id'], g_fife.nodes[n]['latitude'], g_fife.nodes[n]['longitude']
    i = list(sepa['station'][:]).index(id)
    rain = sepa['rainfall_amount'][1, i]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_network)
        
# add the edges
for n, m in g_fife.edges():
    ps = [(g_fife.nodes[n]['latitude'], g_fife.nodes[n]['longitude']), (g_fife.nodes[m]['latitude'], g_fife.nodes[m]['longitude'])]
    tip = 'Distance {d:.2f}km'.format(d=g_fife.edges[n, m]['distance'] / 1000)
    folium.PolyLine(ps, color='red', tooltip=tip, weight=2, opacity=1).add_to(fife_stations_network)

In [214]:
fife_stations_network

### Constructing the Voronoi cells

Interpolation works by mapping observed values through the Voronoi cells (also known as the Thiessen polygon in geographical systems).

In [218]:
stations = fife_stations[['name', 'id', 'geometry']]
stations.crs = 'EPSG:4326'

boundary = GeoDataFrame(columns=['geometry'])
boundary.loc[len(boundary.index)] = {'geometry': fife['geometry']}
boundary.crs = 'EPSG:4326'

cell_centres = points_to_coords(stations['geometry'])
boundary_shape = cascaded_union(boundary['geometry'])

In [219]:
voronoi_cells, voronoi_centres = voronoi_regions_from_coords(cell_centres, boundary_shape)

In [221]:
fife_cells = GeoDataFrame(columns=['geometry'])
for c in voronoi_cells.values():
    fife_cells.loc[len(fife_cells.index)] = {'geometry': c}

In [222]:
fife_stations_voronoi = folium.Map(location=proj.transform(350608, 716790))
folium.GeoJson(fife['geometry']).add_to(fife_stations_voronoi)

# add the stations
for i in range(len(fife_stations)):
    s = fife_stations.iloc[i]
    name, id, lon, lat = s['name'], s['id'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(id)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_voronoi)
    
# add the Voronoi cells
for c in fife_cells['geometry']:
    folium.GeoJson(c).add_to(fife_stations_voronoi)

In [223]:
fife_stations_voronoi

### Constructing the natural neighbour interpolation 

In [ ]:
interpolated_rainfall = []
synthetic_stations = fife_stations.copy()
p = len(synthetic_stations.index)
fife_points_interpolated_rainfall = fife_points.copy()
for i in range(len(fife_points_interpolated_rainfall)):
    s = fife_points_interpolated_rainfall.iloc[i]
    
    # create new cell around the point
    synthetic_stations.loc[p] = {'node': None,
                                 'id': -1,
                                 'geometry': s['geometry']}
    
    # re-compute the Voronoi cells
    synthetic_cell_centres = points_to_coords(synthetic_stations['geometry'])
    synthetic_voronoi_cells, synthetic_voronoi_centres = voronoi_regions_from_coords(synthetic_cell_centres, boundary_shape)

    # get the synthetic cell
    synthetic_cell = None
    for synthetic_cell_index in synthetic_voronoi_centres.keys():
        if p in synthetic_voronoi_centres[synthetic_cell_index]:
            synthetic_cell = synthetic_voronoi_cells[synthetic_cell_index]
            break
    
    # compute intersections with "real" cells
    synthetic_cell_area = synthetic_cell.area
    synthetic_rainfall = 0
    for j in voronoi_centres.keys():
        if j == synthetic_cell_index:
            continue
        area = voronoi_cells[j].intersection(synthetic_cell).area
        if area > 0:
            # there is intersection, add weighted value of observation
            id = g_fife.nodes[synthetic_voronoi_centres[j][0]]['id']
            i = list(sepa['station'][:]).index(id)
            synthetic_rainfall += (area / synthetic_cell_area) * sepa['rainfall_amount'][1, i]
            
    # store synthetic rainfall
    interpolated_rainfall.append(synthetic_rainfall)
    
fife_points_interpolated_rainfall['rainfall'] = interpolated_rainfall

In [257]:
r_max = max(fife_points_interpolated_rainfall['rainfall'])
r_min = min(fife_points_interpolated_rainfall['rainfall'])
r_step = 1 / (r_max - r_min)
fife_points_interpolated_rainfall['normalised_rainfall'] = (fife_points_interpolated_rainfall['rainfall'] - r_min) * r_step

In [258]:
fife_synthetic_cells = GeoDataFrame(columns=['geometry'])
for c in synthetic_voronoi_cells.values():
    fife_synthetic_cells.loc[len(fife_synthetic_cells.index)] = {'geometry': c}

fife_stations_interpolation = folium.Map(location=proj.transform(350608, 716790))
folium.GeoJson(fife['geometry']).add_to(fife_stations_interpolation)

# add the stations
for i in range(len(fife_stations)):
    s = fife_stations.iloc[i]
    name, id, lon, lat = s['name'], s['id'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(id)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_interpolation)

# add the Voronoi cell boundaries
for c in fife_cells['geometry']:
    folium.GeoJson(c, style_function=lambda f: {'fill': False}).add_to(fife_stations_interpolation)
    
# add the heat map
_ = folium.plugins.HeatMap(data=fife_points_interpolated_rainfall[['latitude', 'longitude', 'normalised_rainfall']], min_opacity=0.01, radius=20, blur=20, max_zoom=5).add_to(fife_stations_interpolation)

In [259]:
fife_stations_interpolation